In [2]:
using DataFrames, CSV, Plots, Random

# DecisionTree

In [5]:
using DecisionTree

┌ Info: Precompiling DecisionTree [7806a523-6efd-50cb-b5f6-3fa6f1930dbb]
└ @ Base loading.jl:1278


In [99]:
model = DecisionTreeRegressor(max_depth=5)

DecisionTreeRegressor
max_depth:                5
min_samples_leaf:         5
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     nothing

In [92]:
data = CSV.read("Data//melb_data.csv", DataFrame);
data = skipmissing(data);

In [93]:
filtered = convert( Array, data[!,[:Rooms, :Bathroom, :Landsize, :Lattitude, :Longtitude] ] );
y = convert( Array, data[!,:Price] );

In [94]:
println( typeof(filtered) )
println( typeof(y) )
#first(filtered,5)
size(filtered)

Array{Float64,2}
Array{Float64,1}


(13580, 5)

In [87]:
@time idx = shuffle!(collect(1:size(filtered)[1]));
sp = convert(Int, length(idx)*0.75)

trainX = filtered[idx[1:sp],:];
testX  = filtered[idx[sp+1:end],:];
trainY = y[idx[1:sp]];
testY  = y[idx[sp+1:end]];
#println( typeof(trainX) )
#size(trainX)
println( "train set size ", size(trainX)[1])
println( "test  set size ", size(testX)[1])

  0.000212 seconds (5 allocations: 106.281 KiB)
train set size 10185
test  set size 3395


In [100]:
@time fit!(model, trainX, trainY)

  0.016438 seconds (217 allocations: 654.297 KiB)


DecisionTreeRegressor
max_depth:                5
min_samples_leaf:         5
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     Decision Tree
Leaves: 32
Depth:  5

In [103]:
@time pred = predict(model, testX);
println( "predicion error MSE ",mean( (pred - testY).^2 ) )

  0.000589 seconds (3.40 k allocations: 477.656 KiB)
predicion error MSE 1.9299135028884872e11


In [104]:
@time predT = predict(model, trainX);
println( "train set prediction error MSE ",mean( (predT - trainY).^2 ) )

  0.001850 seconds (10.19 k allocations: 1.399 MiB)
train set prediction error MSE 1.8656336027172964e11


In [95]:
@time mean((pred-testY).^2)

  0.076358 seconds (106.35 k allocations: 5.577 MiB, 31.77% gc time)


1.34781966574143e11

# MLJ

In [82]:
using MLJ

┌ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1278


# ScikitLearn

In [97]:
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split

In [98]:
trainX1, testX1, trainY1, testY1 = train_test_split(filtered, y, test_size=0.25, random_state=42);